In [2]:
import numpy as np
import matplotlib.pyplot as plt 

from scipy import stats
from sklearn import datasets
from sklearn.semi_supervised import label_propagation

from sklearn.metrics import confusion_matrix, classification_report 


In [5]:
digits = datasets.load_digits()
rng = np.random.RandomState(0) # sets the seed()
indices = np.arange(len(digits.data))  # provides equally spaced intervals between the length of the data digits
rng.shuffle(indices)

print(rng)
print(indices)
print(rng.shuffle(indices))


[1081 1707  927 ..., 1653  559  684]
None


In [ ]:

X = digits.data[indices[:330]]
y = digits.target[indices[:330]]
images = digits.images[indices[:330]]

n_total_samples = len(y)
n_labeled_points = 30

indices = np.arange(n_total_samples)

unlabeled_set = indices[n_labeled_points:]

# shuffle everything around
y_train = np.copy(y)
y_train[unlabeled_set] = -1